# Selects

In [295]:
from datetime import datetime
from sqlalchemy import ForeignKey, create_engine, func, insert
from sqlalchemy.orm import Mapped, mapped_column, DeclarativeBase, MappedAsDataclass


class Base(MappedAsDataclass, DeclarativeBase):
    pass


class User(Base):  # class can be called whatever we want, no impact on the table name
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True, init=False)
    name: Mapped[str]
    full_name: Mapped[str | None]
    created_at: Mapped[datetime] = mapped_column(init=False, server_default=func.now())


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))


engine = create_engine("sqlite://", echo=True)
with engine.begin() as conn:
    Base.metadata.create_all(conn)
    conn.execute(
        insert(User),
        [
            {"name": "Sandy", "full_name": "Sandy Kilo"},
            {"name": "Gary", "full_name": "Gary Gareau"},
        ],
    )
    conn.execute(
        insert(Address),
        [
            {"email_address": "sandy@example.com", "user_id": 1},  # User ID 1 is Sandy
            {"email_address": "gary@example.com", "user_id": 2},  # User ID 2 is Gary
            {"email_address": "gary@other.com", "user_id": 2},
        ],
    )

2024-12-09 11:15:44,511 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 11:15:44,512 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-09 11:15:44,512 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 11:15:44,513 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-12-09 11:15:44,513 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 11:15:44,513 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-12-09 11:15:44,514 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 11:15:44,514 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-12-09 11:15:44,515 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 11:15:44,515 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	full_name VARCHAR, 
	created_at DATETIME DEFAULT (CURRENT_TIMESTAMP) NOT NULL, 
	PRIMARY KEY (id)
)


2024-12-09 11:15:44,515 INFO sqlalchemy.engine.Engine [no key 0.0

### `SELECT statements`

In [296]:
from sqlalchemy import text

stmt = text("SELECT name FROM user_account")
with engine.begin() as conn:
    for row in conn.execute(stmt):
        print(row.name)

2024-12-09 11:15:44,528 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 11:15:44,530 INFO sqlalchemy.engine.Engine SELECT name FROM user_account
2024-12-09 11:15:44,530 INFO sqlalchemy.engine.Engine [generated in 0.00026s] ()
Sandy
Gary
2024-12-09 11:15:44,530 INFO sqlalchemy.engine.Engine COMMIT


- Instead of using SQLAlchemy's Core expression language, let use a construct called `select()`
- As we are using ORM-Centric table metadata, the class-bound attributes on ORM models represent SQL Columns, such as `User.name` below

In [297]:
from sqlalchemy import select

print(User)  # User is class, not an instance
print(type(User.name))

with engine.begin() as conn:
    stmt = select(User.name)  # select the 'name' column from the 'user_account' table
    print(stmt)
    for row in conn.execute(stmt):
        print(row.name)

<class '__main__.User'>
<class 'sqlalchemy.orm.attributes.InstrumentedAttribute'>
2024-12-09 11:15:44,537 INFO sqlalchemy.engine.Engine BEGIN (implicit)
SELECT user_account.name 
FROM user_account
2024-12-09 11:15:44,539 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account
2024-12-09 11:15:44,539 INFO sqlalchemy.engine.Engine [generated in 0.00037s] ()
Sandy
Gary
2024-12-09 11:15:44,539 INFO sqlalchemy.engine.Engine COMMIT


- `User.name` is a Python **descriptor** object (object that have custom `__get__`, `__set__`, `__delete__`). This descriptor object is responsible for managing access, assignment, and deletion of the corresponding attribute for instances of the User class (properties in Python are descriptors).

- The arguments we send to `select()` are a series of columns, tables (or a corresponding mapped class), or other so-called "selectables" like aliases or subqueries, and SQL expressions.

- Examples of `select()`: 

In [298]:
# SELECT from a whole table
print(select(User))

SELECT user_account.id, user_account.name, user_account.full_name, user_account.created_at 
FROM user_account


In [299]:
# SELECT from a series of columns
print(select(User.id, User.name, User.created_at))

SELECT user_account.id, user_account.name, user_account.created_at 
FROM user_account


In [300]:
# SELECT from a series of tables/columns from more than on table
# Note: no JOIN clause => it will build a Cartesian Product (if 3 rows in User and 4 in Adress, it will provide 3*4=12 results). This is typically undesirable because it produces many more rows than intended.
stmt = select(User.name, User.full_name, Address.email_address)
print(stmt)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.name:15} {row.full_name:25} {row.email_address}")

SELECT user_account.name, user_account.full_name, address.email_address 
FROM user_account, address
2024-12-09 11:15:44,559 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 11:15:44,560 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.full_name, address.email_address 
FROM user_account, address
2024-12-09 11:15:44,561 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ()
Sandy           Sandy Kilo                sandy@example.com
Sandy           Sandy Kilo                gary@example.com
Sandy           Sandy Kilo                gary@other.com
Gary            Gary Gareau               sandy@example.com
Gary            Gary Gareau               gary@example.com
Gary            Gary Gareau               gary@other.com
2024-12-09 11:15:44,561 INFO sqlalchemy.engine.Engine ROLLBACK


/tmp/ipykernel_86982/3574977805.py:6: SAWarning: SELECT statement has a cartesian product between FROM element(s) "address" and FROM element "user_account".  Apply join condition(s) between each element to resolve.
  for row in conn.execute(stmt):


When selecting from multiple tables, we usualy want to `JOIN` them together. A straightforward way to do this is to use the `select().join_from()` method.

In [301]:
stmt = select(User.name, User.full_name, Address.email_address).join_from(User, Address)
print(stmt)

SELECT user_account.name, user_account.full_name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


- Note that the `join_from()` will normally generate the `ON` criteria based on the presence of `ForeignKey` construct in table metadata.
- The `Adress.user_id` which links to `ForeignKey("user_account.id")` gives `join_from()` what it needs to know.

In [302]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.name:15} {row.full_name:25} {row.email_address}")

2024-12-09 11:15:44,574 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 11:15:44,575 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.full_name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2024-12-09 11:15:44,576 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ()
Sandy           Sandy Kilo                sandy@example.com
Gary            Gary Gareau               gary@example.com
Gary            Gary Gareau               gary@other.com
2024-12-09 11:15:44,577 INFO sqlalchemy.engine.Engine ROLLBACK


- Finally, there's a whole word of more complex `SELECT` statements using aliases, subqueries, etc.
- Such as, "select user names that have more than one email address".

In [303]:
# This subquery calculates the number of email addresses associated with each user (user_id)
# and filters for users who have more than one email address.
email_address_count = (
    select(Address.user_id, func.count(Address.email_address).label("email_count"))
    # Groups the results by user_id so that the COUNT function operates within each user group.
    .group_by(Address.user_id)
    # Filters the results to include only users with more than one email address.
    .having(func.count(Address.email_address) > 1)
    # Converts this query into a subquery that can be used in other queries.
    .subquery()
)

# Result of this subquery is a table with 2 columns: 'user_id' and 'email_count'

stmt = select(User.name, email_address_count.c.email_count).join_from(
    User, email_address_count
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.name} has {row.email_count} email addresses.")

2024-12-09 11:15:44,587 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 11:15:44,589 INFO sqlalchemy.engine.Engine SELECT user_account.name, anon_1.email_count 
FROM user_account JOIN (SELECT address.user_id AS user_id, count(address.email_address) AS email_count 
FROM address GROUP BY address.user_id 
HAVING count(address.email_address) > ?) AS anon_1 ON user_account.id = anon_1.user_id
2024-12-09 11:15:44,589 INFO sqlalchemy.engine.Engine [generated in 0.00215s] (1,)
Gary has 2 email addresses.
2024-12-09 11:15:44,590 INFO sqlalchemy.engine.Engine ROLLBACK


### WHERE clause



In [304]:
print(User.name == "Dave")
User.name == "Dave"

user_account.name = :name_1


- Above, the Python `__eq__()` operator was overriden to produce an expression object
- Instead of returning a `bool`, it returns a `BinaryExpression` object
- The generated expression use bound parameters: SQLAlchemy doesn't immediately insert  `Dave` into the SQL string. Instead, it generates a placeholder (e.g., `:name_1`) and binds the actual value (`Dave`) separately, during `Connection.execute()`.
- for demonstration, they can be seen using a utility methd called `.compile()`:

In [305]:
print((User.name == "Dave").compile(compile_kwargs={"literal_binds": True}))

user_account.name = 'Dave'


### Limiting rows with WHERE criteria

**less than, greater than**

In [306]:
from typing import Any


def print_compiled(something: Any) -> None:
    """Helper to see the full expression when priting it"""
    print(something.compile(compile_kwargs={"literal_binds": True}))

In [307]:
print_compiled((User.id <= 1))

user_account.id <= 1


**fancy string operators**

In [308]:
print_compiled(User.name.icontains("av"))

lower(user_account.name) LIKE '%' || lower('av') || '%'


**`IN` expressions**

In [309]:
print_compiled(User.name.in_(["Rob", "Tim, Walt"]))  # 'in' is a reserved word

user_account.name IN ('Rob', 'Tim, Walt')


- We can add these expressions as `WHERE` criteria using the `.where()` method:

In [310]:
stmt = select(User.name).where(User.name.in_(["Sandy", "Dave", "Rob"]))
print_compiled(stmt)

SELECT user_account.name 
FROM user_account 
WHERE user_account.name IN ('Sandy', 'Dave', 'Rob')


- `where()` can be called multiple times in chaining; criteria is joined by `AND`:

In [311]:
stmt = stmt.where(User.id > 1)
print_compiled(stmt)

SELECT user_account.name 
FROM user_account 
WHERE user_account.name IN ('Sandy', 'Dave', 'Rob') AND user_account.id > 1


- expressions also go into other methods like `ORDER BY` via the `.order_by()` method, still with chaining.

In [312]:
stmt = stmt.order_by(User.id)
print_compiled(stmt)

SELECT user_account.name 
FROM user_account 
WHERE user_account.name IN ('Sandy', 'Dave', 'Rob') AND user_account.id > 1 ORDER BY user_account.id


- expressions can also be selected; below we add a column expression:

In [317]:
from sqlalchemy import literal

# Combines a literal value (a constant string) with a column value from the database using SQLAlchemy,
# creating a new column in the query result
stmt = select(User.name).add_columns(literal("full name: ") + User.full_name)

with engine.connect() as conn:
    for name, generated_full_name in conn.execute(stmt):
        print(f"{name:15}, {generated_full_name}")

2024-12-09 11:16:59,009 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 11:16:59,010 INFO sqlalchemy.engine.Engine SELECT user_account.name, ? || user_account.full_name AS anon_1 
FROM user_account
2024-12-09 11:16:59,011 INFO sqlalchemy.engine.Engine [cached since 40.28s ago] ('full name: ',)
Sandy          , full name: Sandy Kilo
Gary           , full name: Gary Gareau
2024-12-09 11:16:59,011 INFO sqlalchemy.engine.Engine ROLLBACK


### Sum up

- The SQL expression language intends to allow **any SQL structure** to be modeled as a Python expression
- SQLAlchemy goes vey far with this concept; topics not covered here include `UPDATE`, `DELETE`, `UNIONs`, `CTEs`, window functions, set-retuning functions, JSON functions, SQL datatypes
- When writing SQLAlchemy SQL expressions, the hope is that one can be thinking in terms of sQL statements, not "translation".
- Using Python objects for SQL expressions allow fluid composability, database agnosticism to a greater or lesser degree.